# Project No-SQL

Answering Questions On MongoDB

### Individual Project 



In [2]:
#!/usr/bin/python
from pymongo import MongoClient
import sqlite3
import json

### Connecting to MongoDb

In [5]:
client = MongoClient('localhost', 27017)
db=client.proj
user=db.user
tweet=db.tweet

### Collections are stored as:
Collection user:
{
	"user_id":,
	"user_name":,
	"followers":,
	"following":,
	"tweet_count":,
	"hashtags":[ ]
}

Collection tweet:
{
	"tweet_id":,
	"user_id":,
	"tweet_contents":,
	"urls":,
	"date":,
	"time":,
	"favourites":,
	"retweets":,
	"hashtags":[ ]
}

### 1. Tags Associated With a Person

In [55]:
def personTags():
    person= raw_input("Enter twitter handle: ")

    result=user.find({"user_id":""+person+""})
    for tag in list(result)[0]['hashtags']:
        print tag
    
personTags()

Enter twitter handle: @ScopeOnline
artificialintelligence
deeplearning
machinelearning
atomation


### 2. What Social Media users are linke other Social Media users

In [48]:
def similarUsers():
    person= raw_input("Enter twitter handle: ")

    result=user.find({"user_id":""+person+""})
    queryHash= set(list(result)[0]['hashtags'])

    Result=set()
    for search in user.find():
        compareHash= set(search['hashtags'])

        resultSet= queryHash.intersection(compareHash)
        a1=float(len(resultSet))
        a2=float(len(queryHash))

        try:
            accr= a1/a2
            # print a1
            if accr > 0.75:
                Result.add(search['user_id'])
        except ZeroDivisionError:
            a=0

    for x in Result:
        print x
    if(len(Result)==0):
        print "None Found"
        
similarUsers()

Enter twitter handle: @ScopeOnline
@Tsundu_Mak
@ScopeOnline
@ArkangelScrap
@Simone_Johnson1
@EricMackC


### 3. What things are Popular in our Domain

In [6]:
def popularThings():
    resultDict= {}
    for search in tweet.find():
        for x in set(search['hashtags']):
            if x in resultDict:
                resultDict[x]= resultDict[x]+1
            else:
                resultDict[x]=1
    sortDict = sorted(resultDict.items(), key=lambda x: x[1], reverse=True)

    for x in range(0,10):
        print sortDict[x][0]
        
popularThings()

ai
machinelearning
bigdata
deeplearning
datascience
iot
neuralnetworks
blockchain
ml
fintech


### 4. What things are Trending in our Domain

In [8]:
def trendingTopics():
    datesel= raw_input("Enter Date: ")
    resultDict={}
    for search in tweet.find({"date":""+datesel+""}):
        for x in set(search['hashtags']):
            if x in resultDict:
                resultDict[x]= resultDict[x]+1
            else:
                resultDict[x]=1
    sortDict = sorted(resultDict.items(), key=lambda x: x[1], reverse=True)

    for x in range(0,10):
        print sortDict[x][0]

trendingTopics()

Enter Date: 2018-02-26
ai
machinelearning
bigdata
deeplearning
datascience
iot
neuralnetworks
iiot
blockchain
automation


### 5. Should I follow someone back?

In [54]:
def shouldIFollow():
    print "Enter your UserName:"
    meUsr= raw_input()
    print "Enter UserName to compare"
    usrTwo= raw_input()

    result=user.find({"user_id":""+meUsr+""})
    queryHash= set(list(result)[0]['hashtags'])

    result=user.find({"user_id":""+usrTwo+""})
    compareSet= set(list(result)[0]['hashtags'])

    resultSet = queryHash.intersection(compareSet)
    a1 = float(len(resultSet))
    a2 = float(len(queryHash))

    try:
        accr = a1 / a2
        if accr > 0.75:
            print "User is "+str(accr*97)+"% matching, you should follow back"
        else:
            print "User is only " + str(accr*97)+"% matching, you should not follow back"

    except ZeroDivisionError:
        print "Users does not Match, Do not Follow"

shouldIFollow()

Enter your UserName:
@ScopeOnline
Enter UserName to compare
@EricMackC
User is 97.0% matching, you should follow back


### 12. What is my Reach ?

In [39]:
import apikeys
import oauth2
import json

# twitter application credentials
consumer_key = apikeys.consumer_key
consumer_secret = apikeys.consumer_secret
access_token_key = apikeys.access_token_key
access_token_secret = apikeys.access_token_secret
# credentials end

# Aunthitication to twitter account/server
def twit_req(url, type, post_body, http_headers):
    http_method = type
    cOnsumer = oauth2.Consumer(key=consumer_key, secret=consumer_secret)
    tOken = oauth2.Token(key=access_token_key, secret=access_token_secret)
    cLient = oauth2.Client(cOnsumer, tOken)
    resp, content = cLient.request(url, method=http_method, body=post_body, headers=http_headers)
    return content


def getRetweeters(idss, mine):
    url= "https://api.twitter.com/1.1/statuses/retweeters/ids.json?id="+idss+"&count=100&stringify_ids=true"
    getJson = twit_req(url, "GET", "", "NONE")
    parsedJson = json.loads(getJson)
    relUsers=0
    count=0
    userSet= set()
    for x in parsedJson['ids']:
        urluser="https://api.twitter.com/1.1/followers/ids.json?cursor=-1&user_id="+x+"&count=5000"
        getDetails=twit_req(urluser, "GET", "", "NONE")
        pJson = json.loads(getDetails)
        # print pJson
        for user in pJson['ids']:
            userSet.add(user)
        if count==5:
            break
        count+=1


    print "Potential reach is "+str(len(userSet)+int(mine))+" accounts"

    

def whatsMyReach():
    tweetId=raw_input("Enter the Tweet id to compare: ")
    values= tweet.find({"tweet_id":tweetId})[0]
    retweets= values["retweets"]
    userAcc= values["user_id"]
    userFollowers= user.find({"user_id":userAcc})[0]["followers"]

    if retweets>0:
        getRetweeters(tweetId, userFollowers)

whatsMyReach()

Enter the Tweet id to compare: 968036597634564096
Potential reach is 18360
